In [1]:
import theano.tensor as T
import theano
import numpy as np
def _log_likelihood(x_vars, means):
    """
    This function computes the symbolic log-likelihood for a diagonal gaussian defined by the given
    means and a fixed sigma.
    :param x_vars:
    :param means:
    :return:
    """
    std = T.fill(T.zeros_like(means), 0.1)
    zs = (x_vars - means)/std
    return -T.sum(T.log(std), axis=-1) - \
            0.5 * T.sum(T.square(zs), axis=-1) - \
            0.5 * means.shape[-1] * np.log(2 * np.pi)
        

Using gpu device 0: GeForce GTX 980 (CNMeM is disabled, CuDNN 4007)


In [2]:
means = T.tensor4('means', dtype=T.config.floatX)
x_0 = T.tensor4('x_0', dtype=T.config.floatX)
log_l = _log_likelihood(x_0, means)

fn = theano.function(inputs=[x_0, means], outputs=[log_l])

In [3]:
means = np.random.rand(1, 10, 8, 2).astype('float32')
x_0 = np.random.rand(1, 10, 8, 2).astype('float32')

rewards = np.random.rand(1, 10, 8).astype('float32')
log_pi = fn(x_0, means)[0]

print log_pi.shape
print rewards.shape

(1, 10, 8)
(1, 10, 8)


In [15]:
surr = -np.mean(log_pi * rewards, axis=(1, 2))
print surr[0]

T.mean([T.grad(surr[b], r_params, disconnected_inputs='ignore') for b in range(1)])

7.4585369531


NameError: name 'r_params' is not defined

In [26]:
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
_srng = RandomStreams(np.random.randint(1, 2147462579))

loc_mean_t = np.random.rand(1, 2).astype('float32')
means = T.fmatrix('means')

loc_t = T.reshape(T.stack([_srng.normal((1, 2), avg=means[b, :], std=0.1) for b in xrange(1)]), (1, 2))

fn = theano.function(inputs=[means], outputs=[loc_t])
print fn(loc_mean_t)[0].shape

(1, 2)


In [5]:
[[1 for i in range(10)] for j in range(20)]

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [1]:
gorda = 5
print gorda + 2

7


In [2]:
x = 1e-10


NameError: name 'E' is not defined

In [10]:
import theano.tensor as T
import theano
import numpy as np

loc_tm1_var = T.fmatrix('loc_tm1')
x_var = T.tensor4('x', dtype=T.config.floatX)

# Helper function
def _rho(loc_tm1_var, x_var):
    """
    This function will be replaced
    :param loc_tm1:
    :param x:
    :return: x_t (n_batch, img_height, img_width) and x_flattened (n_batch, img_height*img_width)
    """

    x_t_i = []
    loc_tm1_clipped = T.clip(loc_tm1_var, -1, 1)
    for b in xrange(1):
        y_start = theano.tensor.cast((1 + loc_tm1_clipped[0][0]) * (250 - 64) / 2,
                                     'int32')
        x_start = theano.tensor.cast((1 + loc_tm1_clipped[0][1]) * (250 - 64) / 2,
                                     'int32')
        img = x_var[b, :, y_start:y_start + 64, x_start:x_start + 64]
        x_t_i.append(img)

    x_t = T.stack(x_t_i, axis=0)
    x_t_flattened = x_t.reshape((1, -1))
    return x_t, x_t_flattened

x_t, x_t_flattened = _rho(loc_tm1_var, x_var)

In [13]:
loc_tm1 = np.random.rand(1, 2).astype('float32')
x = np.random.rand(1, 4, 250, 250).astype('float32')

fn = theano.function(inputs=[loc_tm1_var, x_var], outputs=[x_t_flattened])
print fn(loc_tm1, x)[0].shape

(1, 16384)


In [30]:
from scipy import stats
npzfile = np.load('../dataset_small_color.npz')
X_train = npzfile['X_train']
y_train = npzfile['y_train']

# Normalize data
X_train = np.transpose(stats.zscore(X_train, axis=0), (0, 3, 1, 2))[:, :3, :, :]

In [31]:
print np.isnan(X_train).any()

True
